# Parse Data

In [1]:
import pandas as pd
import gzip
import json
import pickle

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [3]:
df_fashion = pd.read_pickle('df_fashion.pkl')
df_elect = pd.read_pickle('df_elect.pkl')
df_cloth = pd.read_pickle('df_cloth.pkl')
df_home = pd.read_pickle('df_home.pkl')

In [6]:
# Read the JSON file line by line and load the objects into a list of dictionaries
data = []
with open('Home_and_Kitchen_5.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))

df_home = pd.DataFrame(data)

# Merging each data with their meta

In [12]:
count_fashion = df_fashion['reviewerID'].value_counts()
count_elect = df_elect['reviewerID'].value_counts()
count_cloth = df_cloth['reviewerID'].value_counts()
count_home = df_home['reviewerID'].value_counts()
# Combine the counts
combined_count = count_fashion.add(count_elect, fill_value=0).add(count_cloth, fill_value=0).add(count_home, fill_value=0)
# Filter the combined_count Series to see only the rows where the count is at least five
at_least_five = combined_count[combined_count >= 20]
#result to a DataFrame
at_least_five_df = at_least_five.reset_index().rename(columns={'index': 'Reviewer_id', 0: 'count'})
print(at_least_five_df)

                 Reviewer_id  reviewerID
0       A0039616ADOZ0KMWQRNX        21.0
1       A0053783WS05FZ73CBKH        25.0
2       A0103849GBVWICKXD4T6        23.0
3       A0131404LSWOFZO68ETE        24.0
4       A0139874ED7NYUB55TSR        20.0
...                      ...         ...
166697         AZZXCFBNEWIBQ       201.0
166698         AZZXJAE2DILET        20.0
166699         AZZY4W8E5AX2K        39.0
166700         AZZYJH0XNZ896        28.0
166701         AZZYW4YOE1B6E        65.0

[166702 rows x 2 columns]


In [4]:
# Count occurrences of 'Reviewer_id' in each DataFrame
count_elect = df_elect['reviewerID'].value_counts()
count_cloth = df_cloth['reviewerID'].value_counts()
count_home = df_home['reviewerID'].value_counts()
# Filter the counts to find customers who appear more than 20 times in each DataFrame
two_times_elect = count_elect[count_elect >= 20].index
two_times_cloth = count_cloth[count_cloth >= 20].index
two_times_home = count_home[count_home >= 20].index
# Find the intersection of the 'Reviewer_id's that meet the criteria for all four DataFrames
common_reviewers = set(two_times_cloth).intersection(two_times_elect, two_times_home)
# Subset each original DataFrame to include only the rows with 'Reviewer_id's that meet the criteria
# subset_fashion = df_fashion[df_fashion['reviewerID'].isin(common_reviewers)]
subset_elect = df_elect[df_elect['reviewerID'].isin(common_reviewers)]
subset_cloth = df_cloth[df_cloth['reviewerID'].isin(common_reviewers)]
subset_home = df_home[df_home['reviewerID'].isin(common_reviewers)]
print("Subset Electronics:\n", subset_elect)
print("Subset Clothing:\n", subset_cloth)
print("Subset Home:\n", subset_home)


Subset Electronics:
          overall vote  verified   reviewTime      reviewerID        asin  \
166          5.0  NaN      True  07 24, 2014  A320NYVVIZSZGY  0594481902   
270          5.0  NaN      True   10 3, 2012  A21SESEJ8E52B7  0764207474   
285          4.0  NaN     False  05 20, 2010  A2JCJJNY43QQIV  0789743035   
319          3.0  NaN     False  05 20, 2010  A3U029B8Z5WGI2  0789743035   
716          5.0  NaN      True  11 21, 2012  A3TAS1AG6FMBQW  0972683275   
...          ...  ...       ...          ...             ...         ...   
6737606      1.0  NaN     False  11 28, 2017  A30H2335OM7RD6  B01H747V80   
6738162      4.0  NaN     False  07 23, 2016  A1JZFGZEZVWQPY  B01HBIETM8   
6738813      5.0  NaN     False  09 30, 2018  A2I5QH4F5FDKK5  B01HETFPK4   
6739480      5.0  NaN     False  12 21, 2016   AMLXY5B95T869  B01HIA63ZA   
6739484      5.0    2     False   12 6, 2016  A3NM1MT3Q2FHXV  B01HIA63ZA   

                                  style   reviewerName  \
166     

In [10]:
subset_elect['reviewerID'].nunique()

670

In [6]:
subset_elect.to_pickle("subset_elect.pkl")

In [7]:
subset_cloth.to_pickle("subset_cloth.pkl")

In [8]:
subset_home.to_pickle("subset_home.pkl")

In [2]:
#Read from pickled subset dfs
subset_elect = pd.read_pickle('subset_elect.pkl')
subset_cloth = pd.read_pickle('subset_cloth.pkl')
subset_home = pd.read_pickle('subset_home.pkl')

In [3]:
df_meta_fashion = getDF('meta_AMAZON_FASHION.json.gz')
df_meta_fashion.to_pickle("df_meta_fashion.pkl")

In [2]:
df_meta_elect = getDF('meta_Electronics.json.gz')
df_meta_elect.to_pickle("df_meta_elect.pkl")

In [3]:
df_meta_cloth = getDF('meta_Clothing_Shoes_and_Jewelry.json.gz')
df_meta_cloth.to_pickle("df_meta_cloth.pkl")

In [5]:
df_meta_home = getDF('meta_Home_and_Kitchen.json.gz')
df_meta_home.to_pickle("df_meta_home.pkl")

In [3]:
#read from pickled
df_meta_elect = pd.read_pickle('df_meta_elect.pkl')

In [4]:
df_meta_cloth = pd.read_pickle('df_meta_cloth.pkl')

In [5]:
df_meta_home = pd.read_pickle('df_meta_home.pkl')

# Combine each customer with meta data

In [6]:
merged_elect = pd.merge(subset_elect, df_meta_elect, on='asin')
merged_elect = merged_elect.drop(['vote','verified', 'image', 'tech1', 'fit'
                                 , 'tech2', 'similar_item', 'imageURL', 'imageURLHighRes', 'main_cat'], axis=1)

In [7]:
merged_cloth = pd.merge(subset_cloth, df_meta_cloth, on='asin')
merged_cloth = merged_cloth.drop(['vote','verified', 'image', 'tech1', 'fit'
                                 , 'tech2', 'similar_item', 'imageURL', 'imageURLHighRes', 'main_cat'], axis=1)

In [8]:
merged_home = pd.merge(subset_home, df_meta_home, on='asin')
merged_home = merged_home.drop(['helpful', 'tech1', 'fit', 'similar_item', 
                                'imageURL', 'imageURLHighRes', 'tech2', 'main_cat'], axis=1)
# Create a new column 'col3' with the content of 'col1'
merged_home['style'] = merged_home['feature']

In [9]:
sorted_elect_columns = sorted(merged_elect.columns)
sorted_cloth_columns = sorted(merged_cloth.columns)
sorted_home_columns = sorted(merged_home.columns)

In [11]:
# Check if the sorted columns in all dataframes are equal
if (sorted_elect_columns == sorted_cloth_columns == sorted_home_columns):
    print("All dataframes have the same columns.")
else:
    print("The dataframes have different columns.")

All dataframes have the same columns.


# Final Dataframe

In [12]:
# Reorder the columns in each dataframe based on the merged_fashion dataframe
merged_cloth = merged_cloth[sorted_elect_columns]
merged_home = merged_home[sorted_elect_columns]
# Concatenate the dataframes
final_df = pd.concat([merged_elect, merged_cloth, merged_home], ignore_index=True)

In [13]:
final_df['reviewerID'].value_counts()

A3OXHLG6DIBRW8    726
AVU1ILDDYW301     691
ADLVFFE4VBT8      654
A23GFTVIETX7DS    596
A22CW0ZHY3NJH8    596
                 ... 
A1TRENCBXLC8AS     58
A3HC4071HNLGLZ     57
A3UWRQZV34509I     57
A1W78VPTNJJOF0     56
A8UM78SMPSHC0      50
Name: reviewerID, Length: 670, dtype: int64

In [14]:
final_df['reviewerID'].nunique()

670

In [16]:
final_df.to_pickle("final_df.pkl")

In [1]:
import pandas as pd
final_df = pd.read_pickle('final_df.pkl')

In [3]:
final_df.head()

,overall,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,domain,...,description,title,also_buy,brand,feature,rank,also_view,date,price,details
0,5.0,"07 24, 2014",A320NYVVIZSZGY,0594481902,NaN,Colleen Marie,This AC adapter worked out just fine and as it...,Gives my NOOKY the juice it needs!,1406160000,elect,...,[#############################################...,Barnes &amp; Noble OV/HB Universal Power Kit f...,[],Barnes &amp; Noble,[Genuine Nook Charger + Genuine USB Charger Ca...,[>#55 in Electronics &gt; eBook Readers &amp; ...,"[059449771X, B00QZXQNNO, B014A9T98K, B00QZXQP5...","November 3, 2012",,NaN
1,5.0,"10 3, 2012",A21SESEJ8E52B7,0764207474,{'Format:': ' Kindle Edition'},tweezle,"When I looked at the cover art, I decided this...",Highly recommended!,1349222400,elect,...,[Two Desperate People--<BR>&#9;&#9;One With Al...,Not a Sparrow Falls,"[0764227289, 0764206680, 0764201670, 071808473...",Visit Amazon's Linda Nichols Page,[],"1,693,481 in Books (","[0764201670, 0764227289, 0764206680]",,$28.58,NaN
2,4.0,"05 20, 2010",A2JCJJNY43QQIV,0789743035,{'Format:': ' Paperback'},Katy Lake,this is a great idea. I know that virtually no...,If you've never owned a Macbook...,1274313600,elect,...,"[<b> </b>, , , ]",My MacBook,[],Visit Amazon's John Ray Page,[],"4,226,393 in Books (","[B001GN5OLK, B004FFVLYU, B07B7VFTN9]",,,NaN
3,3.0,"05 20, 2010",A3U029B8Z5WGI2,0789743035,{'Format:': ' Paperback'},KttyKat16,My MacBook is a good step-by-step manual on th...,"Nothing You Can't Find Online, but Easy to Fol...",1274313600,elect,...,"[<b> </b>, , , ]",My MacBook,[],Visit Amazon's John Ray Page,[],"4,226,393 in Books (","[B001GN5OLK, B004FFVLYU, B07B7VFTN9]",,,NaN
4,5.0,"11 21, 2012",A3TAS1AG6FMBQW,0972683275,NaN,Jeri Zerr,The VideoSecu line of articulating wall mounts...,"High quality construction, comes with plenty o...",1353456000,elect,...,[The videosecu TV mount is a mounting solution...,"VideoSecu 24"" Long Arm TV Wall Mount Low Profi...","[B000WYVBR0, B003O1UYHG, B002YV4WJS, B071HW7GS...",VideoSecu,"[Fits most 22"" to 47"" HDTV and some up to 55"" ...",[>#176 in Electronics &gt; Accessories &amp; S...,[],"February 25, 2007",$34.99,NaN


In [502]:
final_df['domain'].value_counts()

elect      55207
clothes    37341
home       18471
Name: domain, dtype: int64

In [2]:
#Inspect duplicate rows
duplicate_rows = final_df[final_df.duplicated(subset='reviewText', keep=False)]
# Display duplicate rows
duplicate_rows.shape

(20765, 21)

In [3]:
#get rid of duplicate rows from final_df
final_df = final_df.drop_duplicates(subset='reviewText', keep='first')
# Reset the index and drop the old index column
final_df.reset_index(drop=True, inplace=True)

In [7]:
final_df

,overall,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,domain,...,description,title,also_buy,brand,feature,rank,also_view,date,price,details
0,5.0,"07 24, 2014",A320NYVVIZSZGY,0594481902,NaN,Colleen Marie,This AC adapter worked out just fine and as it...,Gives my NOOKY the juice it needs!,1406160000,elect,...,[#############################################...,Barnes &amp; Noble OV/HB Universal Power Kit f...,[],Barnes &amp; Noble,[Genuine Nook Charger + Genuine USB Charger Ca...,[>#55 in Electronics &gt; eBook Readers &amp; ...,"[059449771X, B00QZXQNNO, B014A9T98K, B00QZXQP5...","November 3, 2012",,NaN
1,5.0,"10 3, 2012",A21SESEJ8E52B7,0764207474,{'Format:': ' Kindle Edition'},tweezle,"When I looked at the cover art, I decided this...",Highly recommended!,1349222400,elect,...,[Two Desperate People--<BR>&#9;&#9;One With Al...,Not a Sparrow Falls,"[0764227289, 0764206680, 0764201670, 071808473...",Visit Amazon's Linda Nichols Page,[],"1,693,481 in Books (","[0764201670, 0764227289, 0764206680]",,$28.58,NaN
2,4.0,"05 20, 2010",A2JCJJNY43QQIV,0789743035,{'Format:': ' Paperback'},Katy Lake,this is a great idea. I know that virtually no...,If you've never owned a Macbook...,1274313600,elect,...,"[<b> </b>, , , ]",My MacBook,[],Visit Amazon's John Ray Page,[],"4,226,393 in Books (","[B001GN5OLK, B004FFVLYU, B07B7VFTN9]",,,NaN
3,3.0,"05 20, 2010",A3U029B8Z5WGI2,0789743035,{'Format:': ' Paperback'},KttyKat16,My MacBook is a good step-by-step manual on th...,"Nothing You Can't Find Online, but Easy to Fol...",1274313600,elect,...,"[<b> </b>, , , ]",My MacBook,[],Visit Amazon's John Ray Page,[],"4,226,393 in Books (","[B001GN5OLK, B004FFVLYU, B07B7VFTN9]",,,NaN
4,5.0,"11 21, 2012",A3TAS1AG6FMBQW,0972683275,NaN,Jeri Zerr,The VideoSecu line of articulating wall mounts...,"High quality construction, comes with plenty o...",1353456000,elect,...,[The videosecu TV mount is a mounting solution...,"VideoSecu 24"" Long Arm TV Wall Mount Low Profi...","[B000WYVBR0, B003O1UYHG, B002YV4WJS, B071HW7GS...",VideoSecu,"[Fits most 22"" to 47"" HDTV and some up to 55"" ...",[>#176 in Electronics &gt; Accessories &amp; S...,[],"February 25, 2007",$34.99,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99821,5.0,"07 17, 2014",A10Y058K7B96C6,B00KSRV3UQ,"[Fully Stainless Steel, Holds 1000ml, or 34oz,...",midnight821,"In a world of bigger is better coffee pots, te...","Durable, Compact, Nice Design",1405555200,home,...,[Enjoy fresh coffee every day with this elegan...,Francois et Mimi Stainless Steel Double Wall F...,[],Francois et Mimi,"[Fully Stainless Steel, Holds 1000ml, or 34oz,...","[>#113,830 in Kitchen & Dining (See Top 100 in...","[B00MMQOZ1U, B076D3L55Y, B00009ADDS, B00009ADD...","June 6, 2014",,{}
99822,5.0,"07 7, 2014",A79372WBA9IJM,B00KSRV3UQ,"[Fully Stainless Steel, Holds 1000ml, or 34oz,...","Miss Kitty ""Timid Observer""","I love my coffee, but admittedly I&#8217;m not...",High scores on functionality and design. I lo...,1404691200,home,...,[Enjoy fresh coffee every day with this elegan...,Francois et Mimi Stainless Steel Double Wall F...,[],Francois et Mimi,"[Fully Stainless Steel, Holds 1000ml, or 34oz,...","[>#113,830 in Kitchen & Dining (See Top 100 in...","[B00MMQOZ1U, B076D3L55Y, B00009ADDS, B00009ADD...","June 6, 2014",,{}
99823,4.0,"06 24, 2014",A3M6TSEV71537G,B00KSRV3UQ,"[Fully Stainless Steel, Holds 1000ml, or 34oz,...",Sibelius,"This 34 ounce brushed, stainless steel French ...","Excellent insulation, stylish on display",1403568000,home,...,[Enjoy fresh coffee every day with this elegan...,Francois et Mimi Stainless Steel Double Wall F...,[],Francois et Mimi,"[Fully Stainless Steel, Holds 1000ml, or 34oz,...","[>#113,830 in Kitchen & Dining (See Top 100 in...","[B00MMQOZ1U, B076D3L55Y, B00009ADDS, B00009ADD...","June 6, 2014",,{}
99824,5.0,"07 2, 2014",A1FDSZISWIXH1A,B00KSRV3UQ,"[Fully Stainless Steel, Holds 1000ml, or 34oz,...","Terry ""tcinsatx@yahoodotcom"

In [9]:
import pandas as pd
import numpy as np

def sample_five_rows(reviewer_id, df, domains):
    sampled_rows = []
    for domain in domains:
        domain_df = df[(df['reviewerID'] == reviewer_id) & (df['domain'] == domain)].sample(n=5, random_state=3)
        sampled_rows.append(domain_df)
    return pd.concat(sampled_rows)

# Filter: to only include reviewerIDs that appear at least 5 times in each domain
reviewer_counts = final_df.groupby(['reviewerID', 'domain']).size().unstack(fill_value=0)
valid_reviewer_ids = reviewer_counts[(reviewer_counts['elect'] >= 5) & (reviewer_counts['clothes'] >= 5) & (reviewer_counts['home'] >= 5)].index

# Randomly select 50 unique reviewerIDs
selected_reviewer_ids = np.random.choice(valid_reviewer_ids, size=5, replace=False)

domains = ['elect', 'clothes', 'home']
sampled_dfs = []

for reviewer_id in selected_reviewer_ids:
    sampled_df = sample_five_rows(reviewer_id, final_df, domains)
    sampled_dfs.append(sampled_df)

subset_df = pd.concat(sampled_dfs)
# Reset the index and drop the old index column
subset_df.reset_index(drop=True, inplace=True)

In [10]:
subset_df.shape

(75, 21)

In [11]:
subset_df['reviewerID'].value_counts()

A2KETRDA2QWM8M    15
A3UKWQS8SRW6IO    15
A1M92239OM8KPO    15
A1S27P7KJO9I96    15
A28Q0JA9H8WZOD    15
Name: reviewerID, dtype: int64

In [12]:
subset_df['reviewerID'].nunique()

5

# final_df cleaning

In [4]:
#copy final_df and exclude some unnecessary columns
cleaned_data = final_df[['reviewerID','reviewerName' ,'unixReviewTime' ,'asin', 'style', 'reviewText', 'overall', 
                       'domain', 'category', 'title', 'brand','date']].copy()


In [5]:
# Remove HTML tags and convert the 'date' column to datetime
cleaned_data['date'] = cleaned_data['date'].str.replace('<[^<]+?>', '')
cleaned_data['date'] = pd.to_datetime(cleaned_data['date'], format='%B %d, %Y', errors='coerce')

/var/folders/rq/1_3v65yd0yn38jrwqf85v40w0000gn/T/ipykernel_96134/1369819350.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned_data['date'] = cleaned_data['date'].str.replace('<[^<]+?>', '')


In [6]:
cleaned_data['date'] # rows are like June 6, 2014
#give nthe number of rows with empty date
cleaned_data['date'].isnull().sum() # 3787
#drop rows with empty date or empty string date or date with only white spaces
cleaned_data = cleaned_data[cleaned_data['date'].notna()]
cleaned_data = cleaned_data[cleaned_data['date'] != '']
cleaned_data = cleaned_data[cleaned_data['date'] != ' ']
#reset index
cleaned_data.reset_index(drop=True, inplace=True)
cleaned_data['date'].isnull().sum() # 0
# convert date to datetime
cleaned_data['date'] = pd.to_datetime(cleaned_data['date'], format='%B %d, %Y')

In [7]:
cleaned_data['date']

0       2012-11-03
1       2007-02-25
2       2011-06-03
3       2011-06-03
4       2009-11-25
           ...    
52714   2014-06-06
52715   2014-06-06
52716   2014-06-06
52717   2014-06-06
52718   2014-06-06
Name: date, Length: 52719, dtype: datetime64[ns]

In [8]:
import pandas as pd
import ast
def clean_style(style):
    # Handle string representation of dictionary or list
    if isinstance(style, str):
        try:
            style = ast.literal_eval(style)
        except (ValueError, SyntaxError):
            return style  # return the original string if it cannot be parsed
    # If it's a dictionary
    if isinstance(style, dict):
        useful_keys = ['Capacity:', 'Size:', 'Color:', 'Style:', 'style:']  # added 'Style:' and 'style:'
        return ', '.join([f'{k} {v}' for k, v in style.items() if k in useful_keys])
    # If it's a list
    elif isinstance(style, list):
        return ', '.join(style)  # you can add more processing here if you want
    # If it's neither a dictionary nor a list, convert it to a string
    else:
        return str(style)
# Apply the function to the 'style' column
cleaned_data['style'] = cleaned_data['style'].apply(clean_style)
# Drop rows where 'style' is an empty string
cleaned_data = cleaned_data[cleaned_data['style'].str.strip() != '']
# Drop rows where string 'nan' is written in the 'style' column
cleaned_data = cleaned_data[cleaned_data['style'].str.strip() != 'nan']

#for category column
def extract_last_element(list):
    return list[-1]

cleaned_data['category'] = cleaned_data['category'].apply(extract_last_element)


In [9]:
import pandas as pd
import re
def remove_text_inside_parentheses(df, column):
    df[column] = df[column].str.replace(r"\(.*?\)", "", regex=True)
    df[column] = df[column].str.replace(r"\[.*?\]", "", regex=True)
    return df
# Usage
cleaned_data = remove_text_inside_parentheses(cleaned_data, 'title')
#replace all commas with symbol - for title column
cleaned_data['title'] = cleaned_data['title'].str.replace(',', ' -')
#truncate title column to include only first 100 characters
cleaned_data['title'] = cleaned_data['title'].str[:48]
#drop rows where reviewText is more than the median length
cleaned_data = cleaned_data[cleaned_data['reviewText'].str.len() <= cleaned_data['reviewText'].str.len().median()]

In [10]:
# the median length in the reviewText column
cleaned_data['reviewText'].str.len().median()

423.0

In [11]:
# see the entire cell for title column
pd.set_option('display.max_colwidth', -1)
cleaned_data['title'][900:950]

/var/folders/rq/1_3v65yd0yn38jrwqf85v40w0000gn/T/ipykernel_96134/2449507623.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)
/var/folders/rq/1_3v65yd0yn38jrwqf85v40w0000gn/T/ipykernel_96134/2449507623.py:3: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  cleaned_data['title'][900:950]


3776    StarTech.com 12in Power Cord Extension - NEMA 5-
3777    StarTech.com 12in Power Cord Extension - NEMA 5-
3778    StarTech.com 12in Power Cord Extension - NEMA 5-
3779    StarTech.com 12in Power Cord Extension - NEMA 5-
3780    Verbatim DVD+R DL 8.5GB 8X Surface - 5pk Jewel C
3781    Verbatim DVD+R DL 8.5GB 8X Surface - 5pk Jewel C
3782    Verbatim DVD+R DL 8.5GB 8X Surface - 5pk Jewel C
3783    Verbatim DVD+R DL 8.5GB 8X Surface - 5pk Jewel C
3784    Verbatim DVD+R DL 8.5GB 8X Surface - 5pk Jewel C
3787    PCT 1-PORT BI-DIRECTIONAL CABLE TV HDTV AMPLIFIE
3788    PCT 1-PORT BI-DIRECTIONAL CABLE TV HDTV AMPLIFIE
3789    PCT 1-PORT BI-DIRECTIONAL CABLE TV HDTV AMPLIFIE
3794    Lowepro Rezo 15                                 
3826    Monster MP OTG400 BK Outlets To Go Power Strip -
3827    Monster MP OTG400 BK Outlets To Go Power Strip -
3828    Monster MP OTG400 BK Outlets To Go Power Strip -
3829    Monster MP OTG400 BK Outlets To Go Power Strip -
3830    Monster MP OTG400 BK Ou

In [12]:
cleaned_data.columns


Index(['reviewerID', 'reviewerName', 'unixReviewTime', 'asin', 'style',
       'reviewText', 'overall', 'domain', 'category', 'title', 'brand',
       'date'],
      dtype='object')

In [236]:
# Input writing

# cleaned_data trim more

In [13]:
#reset index
cleaned_data.reset_index(drop=True, inplace=True)

In [14]:
#drow the rows where the string '</span>' is written
cleaned_data = cleaned_data[~cleaned_data['style'].str.contains('</span>')]

In [15]:
# get the average length of the style column
cleaned_data['style'].str.len().mean()
# number of rows where the length of the style column is more than the average length
cleaned_data[cleaned_data['style'].str.len() > cleaned_data['style'].str.len().mean()] # 4981 rows
# drop rows where the length of the style column is more than the average length
cleaned_data = cleaned_data[cleaned_data['style'].str.len() <= cleaned_data['style'].str.len().mean()]
# replace all commas in the style column with empty space
cleaned_data['style'] = cleaned_data['style'].str.replace(',', '')

In [16]:
# see the category column where string inches is written
pd.set_option('display.max_colwidth', -1)
cleaned_data[cleaned_data['category'].str.contains('inches')]
# drop rows where string inches is written in the category column
cleaned_data = cleaned_data[~cleaned_data['category'].str.contains('inches')]

/var/folders/rq/1_3v65yd0yn38jrwqf85v40w0000gn/T/ipykernel_96134/2876926659.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [17]:
# replace the commas in the category column with empty space
cleaned_data['category'] = cleaned_data['category'].str.replace(',', '', regex=False)

In [18]:
# replace all commas in brand and title columns with empty space
cleaned_data['brand'] = cleaned_data['brand'].str.replace(',', '', regex=False)
cleaned_data['title'] = cleaned_data['title'].str.replace(',', '', regex=False)

In [19]:
# get the title average length
cleaned_data['title'].str.len().mean()
# see rows where the length of the title column is more than the average length
cleaned_data[cleaned_data['title'].str.len() > cleaned_data['title'].str.len().mean()][100:150]
# for these rows, truncate the title column to include only the string before the symbol -
cleaned_data['title'] = cleaned_data['title'].str.split('-').str[0]

In [20]:
# reset the index and drop the old index column
cleaned_data.reset_index(drop=True, inplace=True)

In [21]:
cleaned_data

,reviewerID,reviewerName,unixReviewTime,asin,style,reviewText,overall,domain,category,title,brand,date
0,A1O97WKID9L3IA,Jeff Wignall,1410739200,B00000J1T1,Color: Blue,"This cable worked out well for me and is a very heavy-duty cable. I was having trouble with wifi on a new Mac Mini and decided to wire it directly to the router. I don't know why, but the Mini just wasn't getting good wifi even though it sits next to the router, but I still needed the router for my laptop and other misc stuff like a blu ray player, so I just used an extra ethernet port in the router and wired the Mini directly--it solved the problem (if you do this, be sure to tell the item that you are hardwiring to shut off the wireless).\n\nWorks great, nice to solve a problem so cheaply and easily.",5.0,elect,Cat 5 Cables,Belkin CAT5e 3,Belkin,2017-04-13
1,A3V1A3C9DTLPME,Peter Faden,1310774400,B00000J1T1,Color: Blue,"I have never bought a Belkin product i wasn't satisfied with. This is no exception. Not that i really give this cord any thought now that it's in use, but that is the point, isn't it? I guess the only other thing to say, is that being blue, it's easy to find in the mass of cords around my TV and computer area.",5.0,elect,Cat 5 Cables,Belkin CAT5e 3,Belkin,2017-04-13
2,A3963R7EPE3A7E,John Schar,1271808000,B00000J1T1,Color: Blue,"I'll make this short and fast much in the same way this cable works. First and foremost: it works and works well. It may be only be 3 feet long and that is totally my fault but it fits exactly as I need it. If I were lucid when I measured the location I would have chosen a longer cord and had a little extra but for what I needed at the time and now the length is perfect. I have a virtual cable factory behind my electronics and when making any changes it is a nightmare because it seems as if everyone will get hung up at the connector but NOT this Belkin. It is snagless and that is fantastic for the lazy or impatient. Those elements alone would be worth buying this cable; however, when the value is added in this is a fantastic buy and I highly recommend it under any circumstances that work.",5.0,elect,Cat 5 Cables,Belkin CAT5e 3,Belkin,2017-04-13
3,A3IUW081KXD3PE,Reviewer,1321228800,B00000J1U5,Style: VideoLink Powerline Internet,"* The setup is as short and simple as the instructions.\n* My wireless capable TV connected to the internet as soon as I selected the wired network setting.\n* Streaming a movie was not what I expected. The video had to re-buffer frequently. I never had this problem using my 802.11g wireless connection.\n\nIt is important to note that the instructions say, ""Do not plug unit into a power strip."" I only have one outlet near my router, and there is a similar Netgear device occupying that outlet. The Netgear device has an outlet so you don't lose an outlet when using this device. I had to plug the Belkin into the Netgear. This is probably violating the ""No power strip rule"". I also was not using the encryption option.",4.0,elect,Computer Cable Adapters,Belkin 8,Belkin,2002-06-01
4,AZMY6E8B52L2T,JP,1350691200,B00000J1EQ,Size: 1-Pack,"Currently I'm moving some family memories from older VHS tapes to DVD-Ram and ripping them to a digital format on my PC. Since these are important memories, I want to have as many backups as possible so rerecording them back onto 'fresh' VHS tapes is something important to me. Comparing these tapes to local price, the price here on Amazon is very competitive. After recording older footage onto these VHS tapes, I'm quite satisfied with the quality of these tapes on the SP record setting. Not much has changed in the VHS tape since it has become such a dinosaur, but it's great to be able to get this product when you need it, for a reasonable price.",5.0,elect,Blank Media,Maxell STD,Maxell,1999-09-04
...,...,...,...,...,...,...,...,...,...,...,...,...
9850,A3NHUQ33CFH3VM,Citizen John,1405555200,B00KSRV3UQ,Fully Stainless Steel Holds 1000ml or 34oz Dishwasher Safe Double wal

# Input writing

In [120]:
import pandas as pd
import numpy as np
def sample_five_rows(reviewer_id, df, domains):
    sampled_rows = []
    for domain in domains:
        domain_specific_df = df[(df['reviewerID'] == reviewer_id) & (df['domain'] == domain)]
        n_samples = min(2, len(domain_specific_df))
        domain_df = domain_specific_df.sample(n=n_samples, random_state=3)
        sampled_rows.append(domain_df)
    return pd.concat(sampled_rows)
# Filter: to only include reviewerIDs that appear at least 1 time in each domain
reviewer_counts = cleaned_data.groupby(['reviewerID', 'domain']).size().unstack(fill_value=0)
valid_reviewer_ids = reviewer_counts[(reviewer_counts['elect'] >= 1) & (reviewer_counts['clothes'] >= 1) & (reviewer_counts['home'] >= 1)].index
# Randomly select 5 unique reviewerIDs
selected_reviewer_ids = np.random.choice(valid_reviewer_ids, size=5, replace=False)
domains = ['elect', 'clothes', 'home']
sampled_dfs = []
for reviewer_id in selected_reviewer_ids:
    sampled_df = sample_five_rows(reviewer_id, cleaned_data, domains)
    sampled_dfs.append(sampled_df)
subset_df = pd.concat(sampled_dfs)
# Reset the index and drop the old index column
subset_df.reset_index(drop=True, inplace=True)

In [121]:
#remove rows where description, title, style, brand, and feature are empty or null or nan
subset_df = subset_df[subset_df['title'].notna()]
subset_df = subset_df[subset_df['title'].notnull()]
subset_df = subset_df[subset_df['title'] != '']
subset_df = subset_df[subset_df['style'].notna()]
subset_df = subset_df[subset_df['style'].notnull()]
subset_df = subset_df[subset_df['style'] != '']
subset_df = subset_df[subset_df['brand'].notna()]
subset_df = subset_df[subset_df['brand'].notnull()]
subset_df = subset_df[subset_df['brand'] != '']

subset_df

In [122]:
import json

def generate_reviewer_text(df):
    df['reviewerName'].fillna(df['reviewerID'], inplace=True)  
    if df['date'].dtype != 'datetime64[ns]': # if the date column is not in datetime format
        df['date'] = pd.to_datetime(df['date'])
    reviewer_texts = []
    # Group by reviewer and iterate over each group
    for reviewer, group in df.groupby('reviewerID'):
        # Sort the group by date
        group = group.sort_values(by='date')       
        reviewer_name = group['reviewerName'].iloc[0]
        items_bought = ', '.join([brand + ' ' + title if brand != title.split(' ')[0] else title for brand, title in zip(group['brand'], group['title'])])       
        categories = ', '.join(group['category'])    
        styles = ', '.join(group['style'].astype(str))
        # Get the longest review text
        longest_review = group.loc[group['reviewText'].str.len().idxmax(), 'reviewText']
        # the text for a reviewer
        text = f"I am {reviewer_name}.\n"
        text += f"I bought the following products: {items_bought}.\n"
        text += f"These products are under the following categories respectively: {categories}.\n"
        text += f"Here are some features of these products respectively: {styles}.\n"
        text += f"For one of the products I bought, I wrote this comment: {longest_review}"
        text = text.replace('\n\n', '\n')
        # a dictionary for each text output
        reviewer_text = {
            "instruction": "Given the products I bought and my comments below, recommend to me what I should buy next.",
            "input": text,
            "output": ""
        }
        reviewer_texts.append(reviewer_text)
    return reviewer_texts

try:
    with open('reviewer_texts.json', 'r') as json_file:
        existing_data = json.load(json_file)
except FileNotFoundError:
    existing_data = []
reviewer_texts = generate_reviewer_text(subset_df)
existing_data.extend(reviewer_texts)

#combined results to the JSON file
with open('reviewer_texts.json', 'w') as json_file:
    json.dump(existing_data, json_file, indent=4)
with open('reviewer_texts.json', 'r') as json_file:
    loaded_texts = json.load(json_file)
# print the texts
for text in loaded_texts:
    print(text["input"])
    print("---------------------------------------------")
    print()

I am Buffy.
I bought the following products: SCI Scandicrafts  Scandicrafts Ceramic Measuring Spoons , Verbatim DVD+R DL 8.5GB 8X Surface , Symphonized NRG Premium Genuine Wood in, instecho Digital Luggage Scale .
These products are under the following categories respectively: Spoons, DVD+R Discs, Earbud Headphones, Luggage Scales.
Here are some features of these products respectively: Measure in sizes of .25 tsp .5 tsp 1 tsp and 1 tbsp Made of Porcelain Dishwasher Safe, Style:  30-Disc, Color:  Red, Color:  Black.
For one of the products I bought, I wrote this comment: I enjoy these spoons because they have lots of style, they're attractive and unique. Since they can break, I use them carefully.
---------------------------------------------

I am SR.
I bought the following products: Belkin 6, OXO Oxo SteeL Kitchen Tool and Utensil Rack, Opsales Polarized Bronze Metal Clip On Flip Up Brown Sun, Opsales Polarized Bronze Metal Clip On Flip Up Brown Sun, Hamilton Beach Hamilton Beach 2, A